<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/File_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# File Search Demo
Beispiel für die Verwendung der OpenAI Assistant API und des File Search-Features.

## Setup

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

## API konfigurieren

In [ ]:
import openai
import os
import time

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)

## Datei hochladen

In [ ]:
# Erstellen eines Vector Stores
vector_store = client.beta.vector_stores.create(name="NeoGeeksVectorStore")

# Pfade zu den Dateien, die hochgeladen werden sollen
dateipfade = ["/content/NeoGeeksAGB.pdf", "NeoGeeksAGB.pdf"]

# Hochladen der Dateien
dateien = [open(dateipfad, "rb") for dateipfad in dateipfade]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=dateien
)

## Assistant erstellen

In [ ]:
assistant = client.beta.assistants.create(
    name="AGB Assistant",
    instructions="Du bist ein Assistant, der Rechtsfragen basierend auf dem bereitgestellten Dokument beantwortet.",
    model="gpt-4-turbo",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

## Thread erstellen

In [ ]:
thread = client.beta.threads.create()

## Anweisung absetzen

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Wie hoch ist das Zahlungsziel von NeoGeeks?"
)

## Run erstellen, ausführen und Ergebnis ausgeben

In [ ]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Abrufen der Antwortnachrichten
messages = client.beta.threads.messages.list(thread.id)
for msg in messages.data:
    if msg.role == "assistant":
        print("Assistant:", msg.content[0].text.value)